## Загрузим нужные библиотеки

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
np.random.seed(17)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder

%matplotlib inline

In [2]:
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import roc_curve,plot_roc_curve, balanced_accuracy_score

In [3]:
df = pd.read_csv("train_dataset_train.csv", low_memory=0)

In [4]:
for c in df.columns:
    print(f'\n\n---{c}---')
    print('major: ')
    print(df[df.label==0][c].value_counts(1).head(5))
    print('\nminor')
    print(df[df.label==1][c].value_counts(1).head(5))
    print('\nall')
    print(df[c].value_counts(1).head(5))



---id---
major: 
6818780    1.715554e-07
6100265    1.715554e-07
7822627    1.715554e-07
3724991    1.715554e-07
2390311    1.715554e-07
Name: id, dtype: float64

minor
610335     0.000006
2195005    0.000006
6863587    0.000006
9973551    0.000006
7404106    0.000006
Name: id, dtype: float64

all
6818780    1.666667e-07
1236699    1.666667e-07
4090073    1.666667e-07
7722817    1.666667e-07
3414179    1.666667e-07
Name: id, dtype: float64


---oper_type + oper_attr---
major: 
1020_-1    0.118855
1022_-1    0.112643
1019_-1    0.070974
8_0        0.069116
1_-1       0.061657
Name: oper_type + oper_attr, dtype: float64

minor
1004_-1    0.652059
2_1        0.280197
2_6        0.060545
1044_-1    0.002959
1020_-1    0.002018
Name: oper_type + oper_attr, dtype: float64

all
1020_-1    0.115526
1022_-1    0.109433
1019_-1    0.068952
8_0        0.067146
1_-1       0.059900
Name: oper_type + oper_attr, dtype: float64


---index_oper---
major: 
102976    0.254599
102971    0.138792
620984 

83318932.0     0.132648
116432632.0    0.055680
7870.0         0.054957
91123247.0     0.045458
8747.0         0.026575
Name: total_qty_oper_login_0, dtype: float64


---total_qty_over_index_and_type---
major: 
132175590.0    0.128287
67549084.0     0.056569
180702765.0    0.052958
159889293.0    0.042214
76146936.0     0.027355
Name: total_qty_over_index_and_type, dtype: float64

minor
132175590.0    0.281314
159889293.0    0.156059
180702765.0    0.148503
156605292.0    0.031355
144324271.0    0.022125
Name: total_qty_over_index_and_type, dtype: float64

all
132175590.0    0.132648
180702765.0    0.055680
67549084.0     0.054957
159889293.0    0.045458
76146936.0     0.026575
Name: total_qty_over_index_and_type, dtype: float64


---total_qty_over_index---
major: 
136819803.0    0.128287
72981134.0     0.056569
188407812.0    0.052958
164927295.0    0.042221
84110459.0     0.027355
Name: total_qty_over_index, dtype: float64

minor
136819803.0    0.281314
164927295.0    0.156088
188407

In [5]:
num_cols = [
    'weight', # Вес в граммах
    'transport_pay', # Общая сумма платы за пересылку
    'weight_mfi', # Суммарная масса вложений
    'price_mfi', # Суммарная стоимость вложений
    'dist_qty_oper_login_1', # Кол-во идент. имен
    'total_qty_oper_login_1', # Кол-во идент. отправлений 
    'total_qty_oper_login_0', # Кол-во неидент. отправлений
    'total_qty_over_index_and_type', # Кол-во отправлений данного mailtype
    'total_qty_over_index', # Общее кол-во отправлений на этом индексе
]

In [6]:
df.describe()

,id,priority,class,weight,mailtype,mailctg,mailrank,directctg,transport_pay,postmark,...,dist_qty_oper_login_1,total_qty_oper_login_1,total_qty_oper_login_0,total_qty_over_index_and_type,total_qty_over_index,is_wrong_sndr_name,is_wrong_rcpn_name,is_wrong_phone_number,is_wrong_address,label
count,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06,6000000.0,6.000000e+06,6.000000e+06,6.000000e+06,...,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06
mean,4.999167e+06,7.499824e+03,3.571923e-01,2.399936e+02,4.995037e+00,7.726733e-01,0.0,1.988388e+00,1.187661e+01,9.069800e-02,...,3.190689e+02,2.775095e+07,2.898887e+07,5.673982e+07,7.178686e+07,2.564167e-03,2.597737e-01,1.709710e-01,1.262833e-03,2.849667e-02
std,2.886491e+06,1.751917e+02,9.861573e-01,4.504002e+02,1.562659e-01,4.407572e-01,0.0,1.073716e-01,2.224794e+01,7.296577e+00,...,4.012494e+02,2.903655e+07,5.334262e+07,7.290388e+07,7.509270e+07,5.057264e-02,4.385104e-01,3.764837e-01,3.551392e-02,1.663869e-01
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.500126e+06,7.503000e+03,0.000000e+00,5.200000e+01,5.000000e+00,1.000000e+00,0.0,2.000000e+00,0.000000e+00,0.000000e+00,...,3.100000e+01,1.113900e+06,1.064800e+04,1.673370e+06,1.040204e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,4.999348e+06,7.503000e+03,0.000000e+00,1.050000e+02,5.000000e+00,1.000000e+00,0.0,2.000000e+00,0.000000e+00,0.000000e+00,...,1.130000e+02,9.570832e+06,3.664710e+05,1.393496e+07,5.270807e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,7.498348e+06,7.506000e+03,0.000000e+00,2.640000e+02,5.000000e+00,1.000000e+00,0.0,2.000000e+00,2.295000e+01,0.000000e+00,...,9.140000e+02,5.610820e+07,8.331893e+07,1.321756e+08,1.368198e+08,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,9.999999e+06,7.506000e+03,5.000000e+00,1.580000e+05,9.000000e+00,5.000000e+00,0.0,2.000000e+00,8.305000e+02,1.024000e+03,...,1.142000e+03,1.416888e+08,4.156581e+08,4.897385e+08,5.002511e+08,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [7]:
df[num_cols].describe()

,weight,transport_pay,weight_mfi,price_mfi,dist_qty_oper_login_1,total_qty_oper_login_1,total_qty_oper_login_0,total_qty_over_index_and_type,total_qty_over_index
count,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06,6.000000e+06
mean,2.399936e+02,1.187661e+01,1.236553e+02,4.560945e+02,3.190689e+02,2.775095e+07,2.898887e+07,5.673982e+07,7.178686e+07
std,4.504002e+02,2.224794e+01,2.084230e+02,2.103291e+03,4.012494e+02,2.903655e+07,5.334262e+07,7.290388e+07,7.509270e+07
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.200000e+01,0.000000e+00,2.300000e+01,5.000000e+01,3.100000e+01,1.113900e+06,1.064800e+04,1.673370e+06,1.040204e+07
50%,1.050000e+02,0.000000e+00,6.000000e+01,1.920000e+02,1.130000e+02,9.570832e+06,3.664710e+05,1.393496e+07,5.270807e+07
75%,2.640000e+02,2.295000e+01,1.230000e+02,4.180000e+02,9.140000e+02,5.610820e+07,8.331893e+07,1.321756e+08,1.368198e+08
max,1.580000e+05,8.305000e+02,1.800000e+04,1.597050e+05,1.142000e+03,1.416888e+08,4.156581e+08,4.897385e+08,5.002511e+08


In [8]:
df[df.label==0].describe()

,id,priority,class,weight,mailtype,mailctg,mailrank,directctg,transport_pay,postmark,...,dist_qty_oper_login_1,total_qty_oper_login_1,total_qty_oper_login_0,total_qty_over_index_and_type,total_qty_over_index,is_wrong_sndr_name,is_wrong_rcpn_name,is_wrong_phone_number,is_wrong_address,label
count,5.829020e+06,5.829020e+06,5.829020e+06,5.829020e+06,5.829020e+06,5.829020e+06,5829020.0,5.829020e+06,5.829020e+06,5.829020e+06,...,5.829020e+06,5.829020e+06,5.829020e+06,5.829020e+06,5.829020e+06,5.829020e+06,5.829020e+06,5.829020e+06,5.829020e+06,5829020.0
mean,4.998968e+06,7.499725e+03,3.362889e-01,2.428926e+02,4.994907e+00,7.840819e-01,0.0,1.988314e+00,1.208624e+01,9.195028e-02,...,3.099055e+02,2.747311e+07,2.807767e+07,5.555078e+07,7.092437e+07,2.392684e-03,2.607958e-01,1.622702e-01,1.136898e-03,0.0
std,2.886562e+06,1.777415e+02,9.565548e-01,4.543314e+02,1.583504e-01,4.336436e-01,0.0,1.077147e-01,2.238752e+01,7.341815e+00,...,3.954022e+02,2.900095e+07,5.326679e+07,7.251879e+07,7.486685e+07,4.885651e-02,4.390688e-01,3.686985e-01,3.369875e-02,0.0
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
25%,2.499864e+06,7.503000e+03,0.000000e+00,5.300000e+01,5.000000e+00,1.000000e+00,0.0,2.000000e+00,0.000000e+00,0.000000e+00,...,3.100000e+01,1.139473e+06,1.064800e+04,1.699222e+06,1.053854e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
50%,4.999382e+06,7.503000e+03,0.000000e+00,1.070000e+02,5.000000e+00,1.000000e+00,0.0,2.000000e+00,0.000000e+00,0.000000e+00,...,1.070000e+02,9.570832e+06,3.387380e+05,1.393496e+07,3.998853e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
75%,7.498126e+06,7.506000e+03,0.000000e+00,2.690000e+02,5.000000e+00,1.000000e+00,0.0,2.000000e+00,2.323000e+01,0.000000e+00,...,9.100000e+02,5.610820e+07,8.331893e+07,1.321756e+08,1.368198e+08,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.0
max,9.999999e+06,7.506000e+03,5.000000e+00,1.580000e+05,9.000000e+00,5.000000e+00,0.0,2.000000e+00,8.305000e+02,1.024000e+03,...,1.142000e+03,1.416888e+08,4.156581e+08,4.897385e+08,5.002511e+08,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.0


In [9]:
bin_cols = ['is_privatecategory', 'is_in_yandex', 'is_return']

In [10]:
# df[bin_cols] = df[bin_cols].replace('N', 0).replace('Y', 1).astype(int)

In [11]:
def parse_index(index_oper):
    if index_oper == ' ':
        return 0
    else:
        return int(float(index_oper))

In [12]:
#некоторые индексы записаны не как целое число (например, 1)
# df['index_oper'] = df['index_oper'].apply(lambda x: parse_index(x))

In [13]:
# стоит стандартизировать имена (например, phone case == Phone Case)
# df['name_mfi'] = df['name_mfi'].apply(lambda x: x.lower())

In [14]:
# некоторые столбцы физически не могут быть равными нулю

In [15]:
index_group = [
    'dist_qty_oper_login_1', # Кол-во идент. имен
    'total_qty_oper_login_1', # Кол-во идент. отправлений 
    'total_qty_oper_login_0', # Кол-во неидент. отправлений
    'total_qty_over_index_and_type', # Кол-во отправлений данного mailtype
    'total_qty_over_index', # Общее кол-во отправлений на этом индексе
]

In [16]:
name_mfi_group = [
    'weight', # Вес в граммах
    'price_mfi', # Суммарная стоимость вложений
]

In [17]:
# df[index_group] = df[index_group].replace(0, np.nan).fillna(
#     df.groupby('index_oper')[index_group].transform('mean')
# ).fillna(0)

In [18]:
# df[name_mfi_group] = df[name_mfi_group].replace(0, np.nan).fillna(
#     df.groupby('name_mfi')[name_mfi_group].transform('mean')
# ).fillna(0)

In [19]:
remove_cols = [
    'id',
    'index_oper',
    'oper_type + oper_attr',
    'type',
    'priority',
    'postmark',
    'name_mfi',
    'mailrank',
    'directctg',
]

In [20]:
cat_cols = [
    'class',
    'mailctg',
    'mailtype',
]

In [21]:
encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(df[cat_cols])

OneHotEncoder(handle_unknown='ignore')

In [22]:
def preprocess_df(df, is_train=False):
    cat_cols = [
        'class',
        'mailctg',
        'mailtype',
    ]
    index_group = [
        'dist_qty_oper_login_1', # Кол-во идент. имен
        'total_qty_oper_login_1', # Кол-во идент. отправлений 
        'total_qty_oper_login_0', # Кол-во неидент. отправлений
        'total_qty_over_index_and_type', # Кол-во отправлений данного mailtype
        'total_qty_over_index', # Общее кол-во отправлений на этом индексе
    ]
    name_mfi_group = [
        'weight', # Вес в граммах
        'price_mfi', # Суммарная стоимость вложений
    ]
    # fill zeroes
    df['index_oper'] = df['index_oper'].apply(lambda x: parse_index(x))
    df['name_mfi'] = df['name_mfi'].apply(lambda x: x.lower())
    df[index_group] = df[index_group].replace(0, np.nan).fillna(
        df.groupby('index_oper')[index_group].transform('mean')
    ).fillna(0)
    
    df[name_mfi_group] = df[name_mfi_group].replace(0, np.nan).fillna(
        df.groupby('name_mfi')[name_mfi_group].transform('mean')
    ).fillna(0)
    df[bin_cols] = df[bin_cols].replace('Y', 1).replace('N', 0).astype(int)
    
    num_not_zero = index_group + name_mfi_group
    
    df[num_not_zero] = df[num_not_zero].replace(0, np.nan)
    df[num_not_zero] = df[num_not_zero].fillna(df[num_not_zero].mean(axis=0))
    

    stay = []
    for col in df.columns:
        if not col in remove_cols:
            stay.append(col)
    df = df[stay]
    
    cats = pd.DataFrame(encoder.transform(df[cat_cols]).toarray()) 
    df = df.join(cats)
    df.drop(cat_cols, axis=1, inplace=True)
    df = df.fillna(0)
    if is_train:
        df = df.drop_duplicates()
    return df    

## Выделим выборки

In [23]:
train_df, val_df = train_test_split(df, stratify=df.label, test_size=0.2, random_state=17)

In [24]:
train_df = preprocess_df(train_df, is_train=True)
val_df = preprocess_df(val_df, is_train=False)

In [25]:
train_df.describe()

,is_privatecategory,is_in_yandex,is_return,weight,transport_pay,weight_mfi,price_mfi,dist_qty_oper_login_1,total_qty_oper_login_1,total_qty_oper_login_0,...,10,11,12,13,14,15,16,17,18,19
count,2.696148e+06,2.696148e+06,2.696148e+06,2.696148e+06,2.696148e+06,2.696148e+06,2.696148e+06,2.696148e+06,2.696148e+06,2.696148e+06,...,2.696148e+06,2.696148e+06,2.696148e+06,2.696148e+06,2.696148e+06,2.696148e+06,2.696148e+06,2.696148e+06,2.696148e+06,2696148.0
mean,6.349800e-04,4.255315e-01,1.239546e-03,2.405785e+02,1.175303e+01,1.272570e+02,5.329540e+02,2.769190e+02,2.605987e+07,2.377705e+07,...,5.563493e-06,1.235095e-03,5.563493e-06,3.490164e-04,2.077037e-05,7.686206e-01,3.708995e-07,1.049646e-04,5.563493e-06,0.0
std,2.519081e-02,4.944234e-01,3.518537e-02,4.414268e+02,2.220451e+01,2.127440e+02,2.161806e+03,3.767106e+02,2.921179e+07,4.859400e+07,...,2.358700e-03,3.512222e-02,2.358700e-03,1.867872e-02,4.557406e-03,4.217144e-01,6.090152e-04,1.024469e-02,2.358700e-03,0.0
min,0.000000e+00,0.000000e+00,0.000000e+00,8.000000e-01,0.000000e+00,0.000000e+00,5.000000e-01,2.777778e-02,2.307692e-01,5.555556e-02,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
25%,0.000000e+00,0.000000e+00,0.000000e+00,5.100000e+01,0.000000e+00,2.500000e+01,1.000000e+02,2.700000e+01,7.758630e+05,8.747000e+03,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
50%,0.000000e+00,0.000000e+00,0.000000e+00,1.040000e+02,0.000000e+00,6.100000e+01,2.700000e+02,9.600000e+01,7.981096e+06,2.135780e+05,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
75%,0.000000e+00,1.000000e+00,0.000000e+00,2.650000e+02,2.287000e+01,1.270000e+02,5.275731e+02,1.940000e+02,5.554931e+07,8.040714e+06,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0
max,1.000000e+00,1.000000e+00,1.000000e+00,1.580000e+05,8.305000e+02,1.800000e+04,1.597050e+05,1.142000e+03,1.416888e+08,4.156581e+08,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.0


In [26]:
X_train = train_df.drop(['label'], axis=1)
y_train = train_df['label']
X_val = val_df.drop(['label'], axis=1)
y_val = val_df['label'] 

In [27]:
np.bincount(y_train) / len(y_train), np.bincount(y_val) / len(y_val)

(array([0.95327593, 0.04672407]), array([0.97150333, 0.02849667]))

## Обучение модели

In [28]:
#билбиотека для работы с дисбалансными данными
from imblearn.ensemble import BalancedRandomForestClassifier, BalancedBaggingClassifier

In [29]:
brf_1 = BalancedRandomForestClassifier(
    n_estimators=300,
    random_state=17,
    verbose=1,
    n_jobs=6,
    class_weight='balanced',
)
 

In [30]:
brf_2 = BalancedRandomForestClassifier(
    n_estimators=300,
    random_state=15,
    verbose=1,
    n_jobs=6,
    class_weight='balanced',
)

In [31]:
brf_1.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:   28.1s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:  2.1min
[Parallel(n_jobs=6)]: Done 300 out of 300 | elapsed:  3.3min finished


BalancedRandomForestClassifier(class_weight='balanced', n_estimators=300,
                               n_jobs=6, random_state=17, verbose=1)

In [32]:
brf_2.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:   28.5s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:  2.5min
[Parallel(n_jobs=6)]: Done 300 out of 300 | elapsed:  4.0min finished


BalancedRandomForestClassifier(class_weight='balanced', n_estimators=300,
                               n_jobs=6, random_state=15, verbose=1)

# ROC, Recall


In [33]:
from sklearn.metrics import recall_score, roc_auc_score
def scoring(y_true, y_pred):
    recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
    roc_auc = roc_auc_score(y_true, y_pred, multi_class='ovo')
    return 0.1 * recall + 0.9 * roc_auc

In [34]:
def treshhold_by_Roc(X_val,y_val,brf):
    fpr, tpr, thresholds = roc_curve(y_val,brf.predict_proba(X_val)[:,1],drop_intermediate=False)
    return thresholds[np.argmin(np.abs(fpr+tpr-1))]

In [35]:
def treshhold_by_score(X_val, y_val, brf):
    pred_proba = brf.predict_proba(X_val)[:,1]
    best_score = -1
    best_t = 0
    for t in np.arange(0, 1, 0.01):
        pred = np.where(pred_proba > t, 1, 0)
        score = scoring(y_val, pred)
        if score > best_score:
            best_score = score
            best_t = t
    return best_t, best_score

In [36]:
tr_1 = treshhold_by_Roc(X_val, y_val, brf_1)
tr_1

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    4.2s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:   18.5s
[Parallel(n_jobs=6)]: Done 300 out of 300 | elapsed:   36.9s finished


0.6198666789919225

In [37]:
tr_r_1, score_1 = treshhold_by_score(X_val, y_val, brf_1)
tr_r_1

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    6.0s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:   27.8s
[Parallel(n_jobs=6)]: Done 300 out of 300 | elapsed:   42.5s finished


0.47000000000000003

In [38]:
tr_2 = treshhold_by_Roc(X_val,y_val, brf_2)
tr_2

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    3.3s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:   17.2s
[Parallel(n_jobs=6)]: Done 300 out of 300 | elapsed:   26.8s finished


0.62

In [39]:
tr_r_2, score_2 = treshhold_by_score(X_val, y_val, brf_2)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    3.7s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:   17.3s
[Parallel(n_jobs=6)]: Done 300 out of 300 | elapsed:   27.6s finished


In [40]:
tr_r_2

0.48

In [41]:
score_2

0.8681552587900854

In [42]:
tr = (tr_1 + tr_2) / 2

In [43]:
tr_r = (tr_r_1 + tr_r_2) / 2

In [44]:
tr

0.6199333394959612

In [45]:
tr_r

0.475

In [46]:
pr_val_proba_1 = brf_1.predict_proba(X_val)[:,1]

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    5.4s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:   24.4s
[Parallel(n_jobs=6)]: Done 300 out of 300 | elapsed:   36.5s finished


In [47]:
pr_val_proba_2 = brf_2.predict_proba(X_val)[:,1]

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    4.0s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:   21.1s
[Parallel(n_jobs=6)]: Done 300 out of 300 | elapsed:   31.2s finished


In [48]:
scoring(y_val, np.where(pr_val_proba_1 > tr_r_1, 1, 0))

0.8681970136425583

In [49]:
scoring(y_val, np.where(pr_val_proba_2 > tr_r_1, 1, 0))

0.8679421006234038

In [50]:
pr_val_proba = (pr_val_proba_1 + pr_val_proba_2)/2

In [51]:
val_pred = np.where(pr_val_proba > tr_r, 1, 0)

In [52]:
scoring(y_val, val_pred)

0.8683866919038711

In [53]:
np.bincount(val_pred) / len(val_pred)

array([0.7857275, 0.2142725])

In [55]:
df = preprocess_df(df, is_train=True)

MemoryError: Unable to allocate 732. MiB for an array with shape (16, 6000000) and data type float64

In [ ]:
X = df.drop(['label'], axis=1)
y = df['label']

In [ ]:
brf_1.fit(X, y)

In [ ]:
brf_2.fit(X, y)

In [ ]:
test_df = pd.read_csv('test_dataset_test.csv', low_memory=1)

In [ ]:
X_test = preprocess_df(test_df, is_train=False)

In [ ]:
test_pred_proba1 = brf_1.predict_proba(X_test)[:, 1]

In [ ]:
test_pred_proba2 = brf_2.predict_proba(X_test)[:, 1]

In [ ]:
t = 0.1 * tr_r + 0.9 * tr

In [ ]:
t

In [ ]:
test_pred_proba = (test_pred_proba1 + test_pred_proba2) / 2

In [ ]:
test_pred = np.where(test_pred_proba1 > t, 1, 0)

In [ ]:
np.bincount(test_pred) / len(test_pred)

In [ ]:
solution = pd.read_csv('sample_solution.csv')
solution['label'] = test_pred
solution.to_csv('brfx2+300+300+w+_tr_060_mean_828.csv', index=False, sep=',', line_terminator='\n')